In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

from statsmodels.tsa.seasonal import seasonal_decompose
# Any results you write to the current directory are saved as output.

# Importing the files

In [ ]:
data = pd.read_csv("../input/all_stocks_5yr.csv")
data.shape

So we have around 619040 rows in the table

In [ ]:
data.columns

In [ ]:
data['Name'].nunique()

We have 505 differents shares data.

In [ ]:
data['Name'].unique()

# Cleaning

For our forecasting algorithms we will be analysing the apple shares i.e. AAPL

In [ ]:
GOOG = data[data['Name'] == 'GOOG']
GOOG.shape

In [ ]:
GOOG.head()

So we daily data from 2014 to 2018 i.e. around 5 years.

In [ ]:
GOOG = GOOG[~(GOOG['date'].apply(lambda x: '2018' in x))]
len(GOOG)

We will be taking the year 2017 as our test period and 2013-2016 as our training period.

In [ ]:
GOOG['date'] = pd.to_datetime(GOOG['date'])
GOOG_train = GOOG.loc[~(GOOG['date'].apply(lambda x: True if x.year == 2017 else False)),['date','open']]
GOOG_test = GOOG.loc[(GOOG['date'].apply(lambda x: True if x.year == 2017 else False)),['date','open']]
GOOG_train.shape,GOOG_test.shape

So our training and testing data set is ready and we will be taking only the opening price everyday.

Lets see how our data looks with some visualisations.

In [ ]:
trace = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

trace1 = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )


data = [trace,trace1]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


![](http://)So now we can see how the stock prices have varied throughout the time 2013 to 2018. We would try to predict the red line. As you can see if you would have invested in Google stock prices in 2013 , you would have already doubled your money in the span 4 years.

> # **Forecasting**

## AutoRegression

This method assumes that the new value as a linear function of the previous observations.

In [ ]:
from statsmodels.tsa.ar_model import AR
from statsmodels.tools.eval_measures import rmse

In [ ]:
GOOG_train_AR = GOOG_train.set_index(['date'])
GOOG_test_AR = GOOG_test.set_index(['date'])
model_AR = AR(GOOG_train_AR['open'])
model_AR = model_AR.fit()

In [ ]:
predicted_AR = model_AR.predict(start = len(GOOG_train_AR), end = len(GOOG_train_AR)+len(GOOG_test_AR)-1)
#predicted_AR

So we have our 251 prediction now. Let us plot it against the actual value.

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = predicted_AR,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


As we can see this almost predict linear values for the stock and is not able capture in trends in the dataset. It only look backs at the few values.

In [ ]:
error_AR = rmse(GOOG_test['open'],predicted_AR)
print('RMSE: ',error_AR)
print('Mean: ',GOOG_test['open'].mean())

This is our first model. Let us see if we can reduce this rmse.

## Moving Average Method

This takes into account the residual errors from the the previous steps, considering a mean process. It then predicts the next value as a lineaer function of this errors

In [ ]:
from statsmodels.tsa.arima_model import ARMA

In [ ]:
GOOG_train_ARMA = GOOG_train.set_index(['date'])
GOOG_test_ARMA = GOOG_test.set_index(['date'])
model_ARMA = ARMA(GOOG_train_AR['open'],order = (0,1))
model_ARMA = model_ARMA.fit()

In [ ]:
predicted_ARMA = model_ARMA.predict(start = len(GOOG_train_AR), end = len(GOOG_train_AR)+len(GOOG_test_AR)-1)
#predicted_ARMA

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = predicted_ARMA,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )




data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_ARMA = rmse(GOOG_test['open'],predicted_ARMA)
print('RMSE: ',error_ARMA)
print('Mean: ',GOOG_test['open'].mean())

This give us a worse result then the Autoregressive method. 

## AutoRegressive Moving Average Method

This method takes into account both the AutoRegressive and the Moving Average Method discussed above.

In [ ]:
GOOG_train_ARMA2 = GOOG_train.set_index(['date'])
GOOG_test_ARMA2 = GOOG_test.set_index(['date'])
model_ARMA2 = ARMA(GOOG_train_AR['open'],order = (2,1))
model_ARMA2 = model_ARMA2.fit()

In [ ]:
predicted_ARMA2 = model_ARMA2.predict(start = len(GOOG_train_AR), end = len(GOOG_train_AR)+len(GOOG_test_AR)-1)
#predicted_ARMA2

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = predicted_ARMA2,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )




data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_ARMA2 = rmse(GOOG_test['open'],predicted_ARMA2)
print('RMSE: ',error_ARMA2)
print('Mean: ',GOOG_test['open'].mean())

As you can see this is able to capture the decreasing trend which simple Autoregressive model was not able to, but it still does not capture the rise of the prices. We have reduced the RMSE but no so much and we will now move onto the Holts Winter Method.

## Holts Winter Method

In [ ]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from matplotlib import pyplot as plt

It has three different types of smoothing i.e. Simple Exponential Smoothing, Double Exponential Smoothing and Triple Exponential Smoothing. Each smoothing takes into consideration a new variable in comparison to other. You can read more about Holts Winter Method [here.](http://otexts.com/fpp2/holt-winters.html)

### Simple Exponential Smoothing

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
GOOG_train_HWSES = GOOG_train.set_index(['date'])
GOOG_test_HWSES = GOOG_test.set_index(['date'])


In [ ]:
alpha = 0.5
model = SimpleExpSmoothing(GOOG_train_HWSES['open']).fit(smoothing_level=alpha,optimized=False)
prediction_HWSES = model.forecast(steps = len(GOOG_test_HWSES))

In [ ]:
train = go.Scatter(
            x = GOOG_train_HWSES.index,
            y = GOOG_train_HWSES['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test_HWSES.index,
            y = GOOG_test_HWSES['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Train'
                  )

predicted = go.Scatter(
            x = GOOG_test_HWSES.index,
            y = prediction_HWSES,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


As we can see this model has increased the prediction by significant amount. Let us now look at the rmse for this model.

In [ ]:
error_HWSES = rmse(GOOG_test_HWSES['open'],prediction_HWSES)
print(error_HWSES)
print('Mean: ',GOOG_test_HWSES['open'].mean())

Let us look the the mean of of our data to see how big a number this RMSE is. As we can see we get an rmse of around 3% of the mean which is good.

### Double Exponential Smoothing

This will take into account the trend that is present in the data

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
#AAPL_HWTES = AAPL.set_index(['date'])
GOOG_train_HWDES = GOOG_train.set_index(['date'])
GOOG_test_HWDES = GOOG_test.set_index(['date'])



In [ ]:

model = ExponentialSmoothing(GOOG_train_HWDES['open'],trend = 'mul').fit() 
prediction_HWDES = model.forecast(len(GOOG_test_HWDES))

In [ ]:
train = go.Scatter(
            x = GOOG_train_HWDES.index,
            y = GOOG_train_HWDES['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test_HWDES.index,
            y = GOOG_test_HWDES['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )

predicted = go.Scatter(
            x = GOOG_test_HWDES.index,
            y = prediction_HWDES,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


The graph seems to tell us that we have almost nailed the prediction as the blue and the green line lmost overlap. You can zoom in hte graph to see that there are actually two lines.

In [ ]:
error_HWDES = rmse(GOOG_test_HWDES['open'],prediction_HWDES)
print('RMSE: ',error_HWDES)
print('Mean: ',GOOG_test_HWDES['open'].mean())

Well only if i can go in the past and invest some money right now 

### Triple Exponential Smoothing

In [ ]:
#AAPL_HWTES = AAPL.set_index(['date'])
GOOG_train_HWTES = GOOG_train.set_index(['date'])
GOOG_test_HWTES = GOOG_test.set_index(['date'])


In [ ]:
model = ExponentialSmoothing(GOOG_train_HWTES['open'],trend = 'add',seasonal = 'add',seasonal_periods = 2).fit() 

"""Considering a monthly seasonal trend. I know stocks usually dont have seasonal trend, 
hence i will expect this sto predict worse then Double Exponential Smoothing"""

prediction_HWTES = model.forecast(len(GOOG_test_HWTES))

In [ ]:
train = go.Scatter(
            x = GOOG_train_HWTES.index,
            y = GOOG_train_HWTES['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test_HWTES.index,
            y = GOOG_test_HWTES['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Train'
                  )

predicted = go.Scatter(
            x = GOOG_test_HWTES.index,
            y = prediction_HWTES,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_HWTES = rmse(GOOG_test_HWTES['open'],prediction_HWTES)
print('RMSE: ',error_HWTES)
print('Mean: ',GOOG_test_HWTES['open'].mean())

## ARIMA

This models takes into account the lag values(AR-p), the difference between the values(I-d) and the error of the previous values(MA-q). The first step in ARIMA is to identify the p,d,q parameters. There are two ways to identify this parameters. 1. Old fashioned way, where we used the PACF,ACF and Dicey-Fuller Test to idetify them. Another way is to sue auto_arima function which would find the parameters for you. We would first use the old way then the auto arima way. You can read how ACF and PACF is used [here.](http://towardsdatascience.com/significance-of-acf-and-pacf-plots-in-time-series-analysis-2fa11a5d10a8)

### ACF

In [ ]:
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf

In [ ]:
plot_acf(GOOG['open'],title='Autocorrelation',lags=40);

This indicaes that there is a AR value to the data. It also seem to indicate that there is no seasonality associated with the data which is expected wih the stocks data. It does not tell us much about the p value.

### PACF

In [ ]:
plot_pacf(GOOG['open'],title='PArtial Autocorrelation',lags=40);

As we see that there is a big drop at 2 on the x axis. So we can say that the value of p will be 2.

### Dickey Fuller Test

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.statespace.tools import diff
from statsmodels.tsa.arima_model import ARIMA,ARIMAResults


In [ ]:

GOOG['d1'] = diff(GOOG['open'],k_diff=1)

GOOGad = adfuller(x = GOOG['d1'].dropna(),autolag='AIC')
GOOGout = pd.Series(GOOGad[0:4],index=['ADF test statistic','p-value','# lags used','# observations'])

for key,val in GOOGad[4].items():
    GOOGout[f'critical value ({key})']=val
print(GOOGout)

The p value now shows that our data is now stationary after 1 difference. And we can use this value for d.

Now let us see if we get the same result using autoarima.

### Auto Arima

In [ ]:
!pip install pmdarima
from pmdarima import auto_arima
import warnings
warnings.filterwarnings("ignore")

In [ ]:
auto_arima(GOOG['open'],seasonal = False,stationary = False,max_p = 4,max_q = 4).summary()

This model gives us the value of p,d,q i.e (0,1,0). Which is the same thing we identified using the PACF,ACF, Dickey Fuller Test. Now the next step is to create the model with the order values.

In [ ]:
model = ARIMA(GOOG_train['open'],order=(3,2,3))  # Trie value different from auto_arima and somehow it gave better results
results = model.fit()
results.summary()

In [ ]:
start=len(GOOG_train)
end=len(GOOG_train)+len(GOOG_test)-1
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels')

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Train'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = predictions,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_ARIMA = rmse(GOOG_test['open'],predictions)
print('RMSE: ',error_ARIMA)
print('Mean: ',GOOG_test['open'].mean())

## SARIMA

Sarima is just an upgrade to ARIMA when we take the Seasonal effect into account.

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


In [ ]:
result = seasonal_decompose(GOOG['open'], model='add',freq = 10)
result.plot();

In [ ]:
auto_arima(GOOG['open'],seasonal=True,m=10).summary()

In [ ]:
model = SARIMAX(GOOG_train['open'],order=(0,1,0),seasonal_order = (2,0,1,10))  
results = model.fit()
results.summary()

In [ ]:
start=len(GOOG_train)
end=len(GOOG_train)+len(GOOG_test)-1
predictions = results.predict(start=start, end=end, dynamic=False, typ='levels')

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Train'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = predictions,
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_SARIMA = rmse(GOOG_test['open'],predictions)
print('RMSE: ',error_SARIMA)
print('Mean: ',GOOG_test['open'].mean())

## LSTM

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_GOOG = sc.fit_transform(GOOG_train[['open']])


In [ ]:
X_train = []
y_train = []
for i in range(60, len(GOOG_train)):
    X_train.append(training_set_GOOG[i-60:i, 0])
    y_train.append(training_set_GOOG[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)


In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout,Flatten


In [ ]:
regressor = Sequential()


In [ ]:
# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(Flatten())

# Adding the output layer
regressor.add(Dense(units = 1))


In [ ]:
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)


In [ ]:
dataset_total = pd.concat((GOOG_train['open'], GOOG_test['open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(GOOG_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [ ]:
X_test = []
for i in range(0,len(GOOG_test)):
    X_test.append(inputs[i:i+60, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#X_test.shape

In [ ]:
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)
GOOG_test['Predicted'] = predicted_stock_price

In [ ]:
train = go.Scatter(
            x = GOOG_train['date'],
            y = GOOG_train['open'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['open'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Train'
                  )

predicted = go.Scatter(
            x = GOOG_test['date'],
            y = GOOG_test['Predicted'],
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_LSTM = rmse(GOOG_test['open'],GOOG_test['Predicted'])
print('RMSE: ',error_LSTM)
print('Mean: ',GOOG_test['open'].mean())

As we can see LSTM has reduced the error drastically and is able to capture a good trend of the stock price.

## Facebook Prophet

In [ ]:
from fbprophet import Prophet

In [ ]:
prophet_train = GOOG_train.copy()
prophet_train.columns = ['ds','y']

In [ ]:
prophet_train['ds'] = pd.to_datetime(prophet_train['ds'])
prophet_train.head()

In [ ]:
model = Prophet()
model.fit(prophet_train)

In [ ]:
future = model.make_future_dataframe(periods = len(GOOG_test),freq = 'B')

In [ ]:
forecast = model.predict(future)
forecast.head()

In [ ]:
forecast[['ds','yhat_lower','yhat_upper','yhat']].head()

In [ ]:
ax = model.plot(forecast);

As we can see it seems to have a good prediction for the training set but not the test set.

In [ ]:
model.plot_components(forecast);

This gives us an indication of the weekly and the yearly trends.

In [ ]:
len(inputs)

In [ ]:
prophet_test = GOOG_test[['date','open']]
prophet_test.columns = ['ds','y']
prophet_test['ds'] = pd.to_datetime(prophet_test['ds'])
prophet_test.head()

In [ ]:
train = go.Scatter(
            x = prophet_train['ds'],
            y = prophet_train['y'],
            mode = 'lines',
            line = dict(
                        color = 'blue'
                       ),
            name = 'Train'
                  )

test = go.Scatter(
            x = prophet_test['ds'],
            y = prophet_test['y'],
            mode = 'lines',
            line = dict(
                        color = 'red'
                       ),
            name = 'Test'
                  )

predicted = go.Scatter(
            x = forecast['ds'],
            y = forecast['yhat'],
            mode = 'lines',
            line = dict(
                        color = 'green'
                       ),
            name = 'Predicted'
                  )

data = [train,test,predicted]

layout = dict(title = 'Google (GOOG) stock prices',
              xaxis = dict(title = 'Year'),
              yaxis = dict(title = 'Stock Price ($)'),
              )

fig = dict(data = data,layout = layout)
iplot(fig)


In [ ]:
error_Prophet = rmse(prophet_test['y'],forecast.iloc[-251:]['yhat'])
print('RMSE: ',error_Prophet)
print('Mean: ',GOOG_test['open'].mean())

In [ ]:
from fbprophet.diagnostics import cross_validation,performance_metrics
from fbprophet.plot import plot_cross_validation_metric

In [ ]:
initial = '100 days'
period = '100 days'
horizon = '20 days'

In [ ]:
df_cv = cross_validation(model,initial = initial,period = period,horizon = horizon)

In [ ]:
df_cv.head()

In [ ]:
performance_metrics(df_cv)

In [ ]:
plot_cross_validation_metric(df_cv,metric = 'rmse');

This can be used to fix the number of days we would like to predict in the future. For this case it is safe to say, we can predict upto 10 days into the future.

In [ ]:
from fbprophet.plot import add_changepoints_to_plot

In [ ]:
fig = model.plot(forecast)
a = add_changepoints_to_plot(fig.gca(),m = model,fcst = forecast)

Now this shows us where there is a sharp change in the trends of the data.

In [ ]:
model_mul = Prophet(seasonality_mode = 'multiplicative')
model_mul.fit(prophet_train)
future = model_mul.make_future_dataframe(periods = len(GOOG_test),freq = 'B')
forecast = model_mul.predict(future)

In [ ]:
fig = model_mul.plot(forecast)
plt.ylim([400,1200])
plt.show()

In [ ]:
model.plot_components(forecast);

In [ ]:
error_Prophet = rmse(prophet_test['y'],forecast.iloc[-251:]['yhat'])
print('RMSE: ',error_Prophet)
print('Mean: ',GOOG_test['open'].mean())

In [ ]:
Thus we seem to have improved from the case when seasonality was additive for the Prophet model.

Thus we seem to have improved from the case when seasonality was additive for the Prophet model.